In [27]:
import pandas as pd
import numpy as np


In [28]:
consolidateSalesReports = pd.read_csv('Consolidate Sales Reports.csv', encoding='latin1')


In [53]:
# numListOfBranch = ['401 Co','402 Co','404 Co','405 Co','412 Co','416 Co',
#                    '417 Co','423 Co', '424 Co','425 Co','426 Co','429 Co','444 Co','490 Co']

monthesList = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
               'September', 'October', 'November', 'December']
listOfBranch = ['ARQOOB SOH','Retailer SOH', 'Final_Cost', 'Month', 'Weeks']

productDetail = ['Part No', "Description", 'UPC']
header = productDetail.copy()
header.extend(listOfBranch.copy())


In [54]:
supplierForecast = pd.DataFrame(columns = list(header))
for col in supplierForecast.columns:
    supplierForecast[col] = consolidateSalesReports[col]


In [55]:
realMonthes = []
lastMonth = ''
for month in monthesList:
    try:
        supplierForecast[month + '_QTY'] = consolidateSalesReports[month[:3] + '_Total']
        realMonthes.append(month + '_QTY')
        lastMonth = month + '_QTY'
    except:
        continue


In [65]:
QTY_Sold =0
for rm in realMonthes:
    QTY_Sold = QTY_Sold + supplierForecast[rm]
supplierForecast['QTY Sold'] = QTY_Sold

avgPerDay = []
avgPerMonth = []
i = 0
for m in supplierForecast['Month']:
    if supplierForecast.loc[i,lastMonth] == 0:
        if m == 0:
            avgPerDay.append(0)
            avgPerMonth.append(0)
        else:
            avgPerDay.append(np.around(QTY_Sold[i] / (m * 30), 2))
            avgPerMonth.append(np.around(QTY_Sold[i] / m,2))
    else:
        avgPerDay.append(np.around(QTY_Sold[i]/((m*30)+(supplierForecast.loc[i,'Weeks']*7)),2))
        avgPerMonth.append(np.around(QTY_Sold[i] / m,2))
    i += 1
supplierForecast['Avh per Day'],supplierForecast['Avg per Month'] = avgPerDay, avgPerMonth

In [66]:
supplierForecast.to_csv('supplierForecast.csv', index=False)